In [1]:
# Dynamic programming solution to the length of the longest common subsequence

def LCS(str1, str2):
    m = len(str1)
    n = len(str2)
    
    # If either string is empty, the length of the LCS is 0
    if m == 0 or n == 0:
        return 0
    
    max_length = 0
    pre = [0] * (n + 1)
    for i in range(1, m + 1):
        current = [0] * (n + 1)
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                current[j] = pre[j - 1] + 1
            else:
                current[j] = max(current[j - 1], pre[j])
            max_length = max(current[j], max_length)
        pre = current
    return max_length

In [2]:
# Different genes to find a relationship tree

a = (0,'TTCTACGGGGGGAGACCTTTACGAATCACACCGGTCTTCTTTGTTCTAGCCGCTCTTTTTCATCAGTTGCAGCTAGTGCATAATTGCTCACAAACGTATC')
b = (1,'TCTACGGGGGGCGTCATTACGGAATCCACACAGGTCGTTATGTTCATCTGTCTCTTTTCACAGTTGCGGCTTGTGCATAATGCTCACGAACGTATC')
c = (2,'TCTACGGGGGGCGTCTATTACGTCGCCAACAGGTCGTATGTTCATTGTCATCATTTTCATAGTTGCGGCCTGTGCGTGCTTACGAACGTATTCC')
d = (3,'TCCTAACGGGTAGTGTCATACGGAATCGACACGAGGTCGTATCTTCAATTGTCTCTTCACAGTTGCGGCTGTCCATAAACGCGTCCCGAACGTTATG')
e = (4,'TATCAGTAGGGCATACTTGTACGACATTCCCCGGATAGCCACTTTTTTCCTACCCGTCTCTTTTTCTGACCCGTTCCAGCTGATAAGTCTGATGACTC')
f = (5,'TAATCTATAGCATACTTTACGAACTACCCCGGTCCACGTTTTTCCTCGTCTTCTTTCGCTCGATAGCCATGGTAACTTCTACAAAGTTC')
g = (6,'TATCATAGGGCATACTTTTACGAACTCCCCGGTGCACTTTTTTCCTACCGCTCTTTTTCGACTCGTTGCAGCCATGATAACTGCTACAAACTTC')

print "The longest common subsequence for string 0 and 2 has length: ", \
    LCS(a[1], c[1])

The longest common subsequence for string 0 and 2 has length:  73


In [3]:
import pandas

genes = [a, b, c, d, e, f, g]
# Get the lengths of the LCS for each pair of strings
n = len(genes)
data = [[0] * n for _ in range(n)]
for i in range(n):
    for j in range(n):
        if i == j:
            data[i][j] = '-'
        
        # If we have already calculated the length of the LCS 
        # for this pair of strings, copy it over
        elif j < i:
            data[i][j] = data[j][i]
        
        else:
            data[i][j] = LCS(genes[i][1], genes[j][1])

# Print a table with the calculated data
genes_list = range(n)
headers = range(n)
pandas.DataFrame(data, genes_list, headers)

,0,1,2,3,4,5,6
0,-,82,73,72,72,70,80
1,82,-,83,81,67,65,70
2,73,83,-,73,62,61,67
3,72,81,73,-,62,60,63
4,72,67,62,62,-,71,82
5,70,65,61,60,71,-,79
6,80,70,67,63,82,79,-


In [4]:
# Dynamic programming solution for the smallest number of changes
# problem. Similar to the length of LCS problem, this also has
# time complexity of O(MN) and space complexity of O(N)
def SNC(str1, str2):
    m = len(str1)
    n = len(str2)
    
    # If either string is empty, the smallest number of changes
    # is the length of the other string, since we have to insert
    # the whole string
    if m == 0 or n == 0:
        return m or n
    
    pre = [0] * (n + 1)
    for i in range(m + 1):
        current = [0] * (n + 1)
        for j in range(n + 1):           
            # If the characters are similar, then no changes needed
            if str1[i-1] == str2[j-1]:
                current[j] = pre[j-1] 
            
            # If the characters are different, consider all
            # possibilities and find minimum
            # To get another character, we can either insert the
            # the needed character, replace the current character
            # with the one needed, or delete the current letter,
            # hoping that a character similar to the one needed
            # will show up.
            else:
                current[j] = 1 + min(current[j-1], # Insert
                                     pre[j],       # Delete
                                     pre[j-1])     # Mutate
        pre = current
    return current[n]

In [5]:
# Calculate the probabilities of mutations, deletions, and insertions

# All parent-child pairs as inferred
gene_pairs = [[a, b], [a, g], [b, c], [b, d], [g, e], [g, f]]

def findProbabilities(parent, child):
    m = len(parent[1])
    n = len(child[1])
    # We can just look up this length in the table, but just in
    # case there is no table available, find the length of LCS.
    l = LCS(parent[1], child[1])
    # Find the smallest number of changes to get from the parent
    # to the child
    total = SNC(parent[1], child[1])
    # Calculate the number of insertions, deletions and
    # mutations
    insNum = total - (m - l)
    delNum = m - n + insNum
    mutNum = total - insNum - delNum
    # Return the probabilities
    return [insNum/float(m), delNum/float(m), mutNum/float(m)]

insProb, delProb, mutProb = 0, 0, 0
for pair in gene_pairs:
    parent, child = pair
    values = findProbabilities(parent, child)
    
    # Average the probabilities from each pair
    insProb += values[0] /float(len(gene_pairs))
    delProb += values[1] /float(len(gene_pairs))
    mutProb += values[2] /float(len(gene_pairs))
    
print "Probability of insertions: ", insProb
print "Probability of deletions: ", delProb
print "Probability of mutations: ", mutProb

Probability of insertions:  0.0622857565012
Probability of deletions:  0.0824615839243
Probability of mutations:  0.0773552009456


In [6]:
# Given a list of genes with no relationships given, build their relationship tree.

class Gene:
    def __init__(self, gene, left = None, right = None, \
                 parent = None):
        self.sequence = gene
        self.p = parent
        self.l_child = left
        self.r_child = right

# Generate the table of the lengths of the LCS for every pair of
# strings.
# Time complexity: O((MN)^2)
# Space complexity: O(N^2)
def LCS_table(genes):
    n = len(genes)
    # Create a n x n table
    data = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i == j:
                data[i][j] = -float('inf')
            # If we have already calculated the length of the LCS
            # for this pair of strings, copy it over
            elif j < i:
                data[i][j] = data[j][i]
            else:
                # Else, calculate the length of the LCS
                # Time complexity to calculate: O(M^2)
                data[i][j] = LCS(genes[i][1], genes[j][1])
    return data

# Find the indexes of the direct relatives
# len(LCS_array) = N, so time complexity is O(N)
def findDirectRelativesIndex(LCS_array, deviation = 3):
    directRelativesIndex = []
    for i in range(len(LCS_array)):
        # If it deviates from the maximum length of LCS by no
        # more than 3, then it is a direct relative
        if abs(max(LCS_array) - LCS_array[i]) <= deviation:
            directRelativesIndex.append(i)
    return directRelativesIndex

# Take the indexes and return an array containing the relatives
# Time complexity also depends on the number of relatives, which 
# is expected to be either 1, 2, or 3, irrespective of N or M.
# Hence, this step should take constant time.
# Therefore, time complexity of this function is O(N), since it
# calls findDirectRelativesIndex() once.
def findDirectRelatives(LCS_array, geneList):    
    directRelatives = []
    directRelativesIndex = findDirectRelativesIndex(LCS_array)
    for i in directRelativesIndex:
        directRelatives.append(geneList[i])
    return directRelatives

# Create a "table," or more exactly a list containing the lists
# of the direct relatives for each gene.
# Each gene has a list of length independent from N or M, so this
# function takes up O(N) space.
# It calls findDirectRelatives() once for each gene, hence time
# complexity of this function is O(N^2)
def relativesTable(geneList):
    n = len(geneList)
    lengthTable = LCS_table(geneList)
    relativesTable = [[] for _ in range(n)]
    nodesList = []
    for i in range(n):
        relativesTable[i] = \
            findDirectRelatives(lengthTable[i], geneList)
    return relativesTable

# This function adds new nodes to the relationship tree. Its 
# time complexity depends on the number of relatives, which 
# does not relate to N or M as mentioned above. This is where
# the implementation differs from the proposed algorithm, using
# two dicts to check which relative is the parent by keeping 
# the index of the strings that have an object already and the 
# corresponding object.
# Because createdList is a dict, it has O(1) lookup time.
# Hence, it takes O(1) time to create pointers.
def createRelatives(node, relativesTable, geneList, createdList, \
                    nodesList):
    index = node.sequence[0]
    for gene in relativesTable[index]:
        # All genes in createdList are at the same level or higher
        # up in the relationship tree, which means that if a direct
        # relative is in this dict, it must be the parent.
        if gene[0] in createdList and node.p == None:
            node.p = nodesList[gene[0]]
        else:
            child = Gene(gene)
            createdList[gene[0]] = gene
            nodesList[gene[0]] = child
            if node.l_child == None:
                node.l_child = child
            else:
                node.r_child = child
    return createdList, nodesList

# The main function
def createTree(geneList):
    n = len(geneList)
    # Creating this table takes O((MN)^2) time and O(N^2)
    # space
    lengthTable = LCS_table(geneList)
    # Creating this table takes O(N) space and O(N^2) time
    # as mentioned above
    r_Table = relativesTable(geneList)
    createdList, nodesList = {}, {}
    # Loop through all genes to find the root takes O(N) time
    for i in range(n):
        if len(r_Table[i]) == 2:
            l = Gene(r_Table[i][0])
            createdList[r_Table[i][0][0]] = r_Table[i][0]
            nodesList[r_Table[i][0][0]] = l
            
            r = Gene(r_Table[i][1])
            createdList[r_Table[i][1][0]] = r_Table[i][1]
            nodesList[r_Table[i][1][0]] = r
            
            root = Gene(geneList[i], l, r)
            createdList[geneList[i][0]] = geneList[i]
            nodesList[geneList[i][0]] = root
    # Creating and adding nodes to the tree takes O(N) time
    # and space
    to_be_done = [root.l_child, root.r_child]
    while to_be_done:
        node = to_be_done.pop()
        createdList, nodesList = createRelatives(node, r_Table, \
                        geneList, createdList, nodesList)
        if node.l_child is not None:
            to_be_done.append(node.l_child)
        if node.r_child is not None:
            to_be_done.append(node.r_child)
    return root

def traverse(root):
    this_gen = [root]
    while this_gen:
        next_gen = []
        for n in this_gen:
            print n.sequence[0],
            if n.l_child is not None:
                next_gen.append(n.l_child)
            if n.r_child is not None:
                next_gen.append(n.r_child)
        print
        this_gen = next_gen

In [7]:
# Traverse the relationship tree
root = createTree(genes)
traverse(root)

0
1 6
2 3 4 5
